In [1]:
import torch
import torch.nn as nn
import numpy as np
from  scipy.ndimage import zoom as imzoom

In [2]:
%matplotlib inline

from PIL import Image
from matplotlib import mlab
import matplotlib.pyplot as plt
import numpy as np


In [3]:
from SpatialTransformer2D import SpatialTransformer2d


In [15]:
ConvST_net = SpatialTransformer2d( num_input_channels = 1,
                 feature_net = None,
                 out_patch_size = 32,
                 out_stride = 32,
                 min_zoom = 1.0,
                 max_zoom = 1.0,
                 min_tilt = 1.0,
                 max_tilt = 1.0,
                 max_rot = 0.0,
                 max_shift = 0.0,
                 mrSize = 1.0)

In [5]:
img = Image.open('/home/old-ufo/Dropbox/Mik/graf/img1.png').convert('RGB')
img = np.sum(np.array(img)/255.0, axis = 2)
#img = imzoom(img, (0.5,0.5))
var_image = torch.autograd.Variable(torch.from_numpy(img.astype(np.float32)))
var_image_reshape = var_image.view(1, 1, var_image.size(0),var_image.size(1))
#var_image_reshape = var_image_reshape.cuda()
#ConvST_net.cuda()




In [6]:
img2 = Image.open('/home/old-ufo/Dropbox/Mik/graf/img2.png').convert('RGB')
img2 = np.sum(np.array(img2)/255.0, axis = 2)
#img = imzoom(img, (0.5,0.5))
var_image2 = torch.autograd.Variable(torch.from_numpy(img2.astype(np.float32)))
var_image_reshape2 = var_image2.view(1, 1, var_image2.size(0),var_image2.size(1))
#var_image_reshape2 = var_image_reshape2.cuda()


In [7]:
out_image,aff_pts = ConvST_net(var_image_reshape)

In [8]:
out_image2,aff_pts2 = ConvST_net(var_image_reshape2)

In [9]:
loss = torch.sum(out_image - out_image2)

print loss

Variable containing:
-5181.1489
[torch.FloatTensor of size 1]



In [10]:
from HardNet import HardNet
hardnet = HardNet()
checkpoint = torch.load('HardNetLib.pth')
hardnet.load_state_dict(checkpoint['state_dict'])


In [11]:
class SparseImgRepresenter(nn.Module):
    def __init__(self, 
             detector_net = None,
             descriptor_net = None,    
             use_cuda = False):
        super(SparseImgRepresenter, self).__init__()
        self.detector = detector_net;
        self.descriptor = descriptor_net;
        return
    def forward(self, input_img):
        aff_norm_patches, LAFs = self.detector(input_img)
        descs = self.descriptor(aff_norm_patches);
        return aff_norm_patches, LAFs, descs

In [16]:
SIRNet = SparseImgRepresenter(detector_net = ConvST_net, descriptor_net = hardnet)

In [17]:
aff_norm_patches, LAFs, descs = SIRNet(var_image_reshape)

In [ ]:
def distance_matrix_vector(anchor, positive):
    """Given batch of anchor descriptors and positive descriptors calculate distance matrix"""

    d1_sq = torch.sum(anchor * anchor, dim=1)
    d2_sq = torch.sum(positive * positive, dim=1)
    eps = 1e-6
    return torch.sqrt((d1_sq.expand(anchor.size(0), anchor.size(0)) + torch.t(d2_sq.expand(positive.size(0), positive.size(0)))
                      - 2.0 * torch.bmm(positive.unsqueeze(0), torch.t(anchor).unsqueeze(0)).squeeze(0))+eps)

def LAFs_to_H_frames(aff_pts):
    H3_x = torch.Tensor([0, 0, 1 ]).unsqueeze(0).unsqueeze(0).expand_as(aff_pts[:,0:1,:]);
    H3_x = torch.autograd.Variable(H3_x)
    return torch.cat([aff_pts, H3_x], dim = 1)
def get_GT_correspondence_indexes(aff_pts1,aff_pts2, H1to2, dist_threshold = 4):
    LHF2 = LAFs_to_H_frames(aff_pts2)
    LHF2_reprojected_to_1 = torch.bmm(H1to2.unsqueeze(0).expand_as(LHF2), LHF2);
    LHF2_reprojected_to_1 = LHF2_reprojected_to_1 / LHF2_reprojected_to_1[:,2:,2:].expand_as(LHF2_reprojected_to_1);
    just_centers1 = aff_pts1[:,:,2];
    just_centers2_repr_to_1 = LHF2_reprojected_to_1[:,0:2,2];
    dist  = distance_matrix_vector(just_centers1, just_centers2_repr_to_1)
    min_dist, idxs_in_2 = torch.min(dist,1)
    plain_indxs_in1 = torch.autograd.Variable(torch.arange(0, idxs_in_2.size(0))).cuda()
    mask =  min_dist <= dist_threshold
    return min_dist[mask], plain_indxs_in1[mask], idxs_in_2[mask]


In [18]:
dists, idxs_in1, idxs_in2 = get_GT_correspondence_indexes(aff_pts, aff_matr_2, H1to2_v, dist_threshold = 5);
#print gt_idxs
def train_SIR_Net_in_UCN_style(img1,img2, H1to2p, SIRNet):
    
    return



Variable containing:
-3.1480e-02 -1.1687e-01  7.4637e-02  ...  -8.0522e-02  5.3022e-02  1.5812e-02
-6.3397e-02 -1.1175e-01  2.4787e-01  ...  -4.0955e-02  1.8381e-02  5.1095e-02
 1.0474e-02 -9.5838e-02  1.0675e-01  ...  -1.1603e-01  1.0846e-01 -4.7470e-02
                ...                   ⋱                   ...                
 8.6725e-02 -2.1020e-02  7.7554e-02  ...   7.2322e-02 -9.1421e-02  1.0271e-01
 8.2311e-02  4.6112e-03  5.9497e-02  ...   2.4793e-02  4.3884e-02 -1.2280e-01
-9.8883e-02 -6.6458e-02  7.0221e-02  ...  -2.0086e-02  1.6380e-03  5.9372e-02
[torch.FloatTensor of size 500x128]